# Import librarys

In [1]:
from keras.models import load_model
from keras.metrics import MeanSquaredError
from sklearn.base import BaseEstimator
from keras.models import Model
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import keras
from keras.callbacks import History
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import keras
from scikeras.wrappers import KerasRegressor
from keras.layers import Input
from keras.optimizers import RMSprop
import yfinance as yf
from matplotlib.backends.backend_pdf import PdfPages
import pickle
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense
from tensorflow.keras.losses import MeanSquaredError
from sklearn.base import BaseEstimator
from sklearn.model_selection import RandomizedSearchCV

# CSV einlesen

In [2]:
def download_stock_data(stock_symbols, start_date, end_date, output_file):
    with open(output_file, 'w') as f:
        # Write the column headings
        f.write('Date,Open,High,Low,Close,Volume\n')
        
        for symbol in stock_symbols:
            # Load the share data for every symbol and the defined time period
            stock_data = yf.download(symbol, start=start_date, end=end_date)
            stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']]  # Auswahl der gewünschten Spalten
            stock_data.to_csv(f, header=False)  # Schreiben der Daten in die Datei


In [3]:
# Define time period and symbol
stock_symbols = ['ADS.DE']
start_date = '2000-01-01'
end_date = '2018-12-31'
output_file = 'stock_data.csv'

# Load and store the share data
download_stock_data(stock_symbols, start_date, end_date, output_file)

# Search and delete empty rows in the CSV file
with open(output_file, 'r') as file:
    lines = file.readlines()

# Filter empty rows
lines = [line.strip() for line in lines if line.strip()]

# Overwrite the file with the cleaned rows
with open(output_file, 'w') as file:
    file.write('\n'.join(lines))

[*********************100%%**********************]  1 of 1 completed


In [4]:
# load CSV
df = pd.read_csv('stock_data.csv')

# transform date to index
df.set_index('Date', inplace=True)

# Show first rows
print(df.head())
print(len(df))

                 Open       High        Low      Close   Volume
Date                                                           
2000-01-03  18.875000  19.125000  17.730000  18.580000   784844
2000-01-04  17.922501  17.937500  17.375000  17.924999  1212496
2000-01-05  17.312500  17.995001  17.137501  17.500000  3316176
2000-01-06  17.922501  18.422501  17.650000  18.250000  3043168
2000-01-07  17.977501  18.400000  17.977501  18.000000  3552880
4865


In [5]:
# Check, if there are any missing numbers
missing_values_before = df.isnull().values.any()

# Treat missing values with the mean of the previous and the following row
for column in df.columns:
    missing_values = df[column].isnull()
    df.loc[missing_values, column] = (df[column].shift() + df[column].shift(-1)) / 2

# Check, if there are missing values after the Treatment
missing_values_after = df.isnull().values.any()

# Output of the missing values before and after the Treatment
print("Fehlende Werte vor der Behandlung gefunden:", missing_values_before)
print("Fehlende Werte nach der Behandlung gefunden:", missing_values_after)

Fehlende Werte vor der Behandlung gefunden: False
Fehlende Werte nach der Behandlung gefunden: False


In [6]:
# show first rows
print(df.head())

                 Open       High        Low      Close   Volume
Date                                                           
2000-01-03  18.875000  19.125000  17.730000  18.580000   784844
2000-01-04  17.922501  17.937500  17.375000  17.924999  1212496
2000-01-05  17.312500  17.995001  17.137501  17.500000  3316176
2000-01-06  17.922501  18.422501  17.650000  18.250000  3043168
2000-01-07  17.977501  18.400000  17.977501  18.000000  3552880


# Data normalization

In [7]:
# Copy the data frame and remove the column "date"
nf = df.copy()

# Remove the index name
nf.index.name = None

# Normalize the data, except the "date" (index)
scaler = MinMaxScaler()
nf_normalized = scaler.fit_transform(nf)

# Create a new data frame with the normalized data and the original index
nf = pd.DataFrame(nf_normalized, columns=nf.columns, index=nf.index)

# # Show the normalized data
print(nf.head())

# Show the length of the data frame
print("Länge des Datensatzes:", len(nf))

                Open      High       Low     Close    Volume
2000-01-03  0.033427  0.032015  0.030441  0.032611  0.008669
2000-01-04  0.028794  0.026235  0.028702  0.029401  0.013393
2000-01-05  0.025827  0.026515  0.027538  0.027318  0.036629
2000-01-06  0.028794  0.028595  0.030049  0.030994  0.033613
2000-01-07  0.029062  0.028486  0.031654  0.029768  0.039243
Länge des Datensatzes: 4865


In [8]:
# save scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# read predicting sequence

In [9]:
# Define time period and output file
stock_symbols = ['BNR.VI']
start_date = '2018-10-01'
end_date = '2018-12-31'
output_file = 'stock_data_pre.csv'

# Load and store the share data
download_stock_data(stock_symbols, start_date, end_date, output_file)

# Search and delete the empty rows in the CSV file
with open(output_file, 'r') as file:
    lines = file.readlines()

# Filter the empty rows
lines = [line.strip() for line in lines if line.strip()]

# Overwrite the file with the cleaned rows
with open(output_file, 'w') as file:
    file.write('\n'.join(lines))

[*********************100%%**********************]  1 of 1 completed


In [10]:
# load CSV
dfPre = pd.read_csv('stock_data_pre.csv')

# Drop empty rows
dfPre.dropna(inplace=True)

# transform date to index
dfPre.set_index('Date', inplace=True)

# Check the number of rows
num_rows = dfPre.shape[0]

# If more than 60 rows, keep the last 60 rows
if num_rows > 60:
    dfPre = dfPre.tail(60)

# Show first rows
print(dfPre.head())

                 Open       High        Low      Close  Volume
Date                                                          
2018-10-03  51.119999  51.119999  51.119999  51.119999       0
2018-10-04  51.119999  51.119999  51.119999  51.119999       0
2018-10-05  51.119999  51.119999  51.119999  51.119999       0
2018-10-08  51.119999  51.119999  51.119999  51.119999       0
2018-10-09  51.119999  51.119999  51.119999  51.119999       0


In [11]:
# Check, if there are missing values before the treatment
missing_values_beforePre = dfPre.isnull().values.any()

# Treat missing values for all columns with the mean of the previous and following row
for column in df.columns:
    missing_valuesPre = dfPre[column].isnull()
    dfPre.loc[missing_valuesPre, column] = (dfPre[column].shift() + dfPre[column].shift(-1)) / 2

# Check, if there are missing values after the treatment
missing_values_afterPre = dfPre.isnull().values.any()

# Output of the missing values before and after the treatment
print("Fehlende Werte vor der Behandlung gefunden:", missing_values_beforePre)
print("Fehlende Werte nach der Behandlung gefunden:", missing_values_afterPre)

# show lenth of data set
print(len(dfPre))

Fehlende Werte vor der Behandlung gefunden: False
Fehlende Werte nach der Behandlung gefunden: False
60


# Data normalization prediction data

In [12]:
# Copy the data frame and remove the column "date"
nfPre = dfPre.copy()

# Remove the index name
nfPre.index.name = None

# Normalize the data, except the "date" (Index)
nfPre_normalized = scaler.fit_transform(nfPre)

# Create a data frame with the normalized data and the original index
nfPre = pd.DataFrame(nfPre_normalized, columns=nfPre.columns, index=nfPre.index)

# Convert DataFrame to NumPy array
nfPre = nfPre.to_numpy()

# Show length of the data frame
print("Länge des Datensatzes:", len(nfPre))


Länge des Datensatzes: 60


# Create Sequences

In [13]:
# Function to create sequenences for LSTM model
def create_sequences(data_input, seq_length):
    data = data_input
    X, y = [], []
    for i in range(len(data) - seq_length - 20):
        X.append(data[i:i+seq_length])
        y.append(data.iloc[i+seq_length+20]) # acess to Close-Value of the 20th row
    return np.array(X), np.array(y)

In [14]:
# create sequence
sequence_length = 60
X, y = create_sequences(nf, sequence_length)

# Split the data in Training, Test and validation

In [15]:
# Split data in training and test (80% training, 20% test)
train_data, test_data = train_test_split(nf, test_size=0.2, shuffle=False)

# Split data in training and validation (80% training, 20% validation)
train_data, val_data = train_test_split(train_data, test_size=0.2, shuffle=False)

# Split the train data in sequences
X_train, y_train = create_sequences(train_data, sequence_length)

# Split the validation data in sequences
X_val, y_val = create_sequences(val_data, sequence_length)

# Split the test data in sequences
X_test, y_test = create_sequences(test_data, sequence_length)

# show training, validation and test sets
print("Trainingsdaten:", len(train_data))
print("Validierungsdaten:", len(val_data))
print("Testdaten:", len(test_data))

Trainingsdaten: 3113
Validierungsdaten: 779
Testdaten: 973


In [16]:
# Define train_size, val_size and test_size
train_size = len(train_data)
val_size = len(val_data)
test_size = len(test_data)

# Extract features and target variables for training, validation, and test sets
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

print(X_train.shape)
print(y_train.shape)

(3113, 60, 5)
(3113, 5)


In [17]:
# Convert the train, val and prediction data into tensors
X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_val_tensor = tf.convert_to_tensor(X_val, dtype=tf.float32)
y_val_tensor = tf.convert_to_tensor(y_val, dtype=tf.float32)

print(type(X_train_tensor))
print(type(y_train_tensor))

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>


# Define model

In [18]:
param_dist = {
    'units': [25, 50, 100, 300, 400, 500, 800],
    'dropout_rate': [0.05, 0.1, 0.2, 0.3, 0.4, 0.5],
    'batch_size': [16, 32, 64, 128, 256],
    'epochs': [10, 20, 30, 40, 50, 80, 100],
    'optimizer': ['adam', 'rmsprop'],
    'learning_rate': [0.001, 0.003, 0.005, 0.008, 0.01],
}

In [19]:
class LSTMModel(BaseEstimator):
    def __init__(self, batch_size=32, optimizer='adam', units=50, dropout_rate=0.2, epochs=10, learning_rate=0.001):
        self.batch_size = batch_size
        self.optimizer = optimizer
        self.units = units
        self.dropout_rate = dropout_rate
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.model = self.create_lstm_model()  # Hier das Modell erstellen
        
    def create_lstm_model(self):
        if self.optimizer == 'adam':
            optimizer = Adam(learning_rate=self.learning_rate)
        elif self.optimizer == 'rmsprop':
            optimizer = RMSprop(learning_rate=self.learning_rate)
        else:
            raise ValueError("Unrecognized optimizer name.")
        input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
        lstm_layer1 = LSTM(units=self.units, return_sequences=True)(input_layer)
        dropout_layer1 = Dropout(self.dropout_rate)(lstm_layer1)
        lstm_layer2 = LSTM(units=self.units, return_sequences=False)(dropout_layer1)
        dropout_layer2 = Dropout(self.dropout_rate)(lstm_layer2)
        output_layer = Dense(units=5)(dropout_layer2)
    
        model = Model(inputs=input_layer, outputs=output_layer)
    
        model.compile(optimizer=optimizer, loss='mse', metrics=[MeanSquaredError()])
    
        return model
        
    def fit(self, X, y):
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size)
        
    def predict(self, X):
        return self.model.predict(X)

In [20]:
lstm_model = LSTMModel()  # Create a instance of the lstm model
random_search = RandomizedSearchCV(estimator=lstm_model,
                                   param_distributions=param_dist,
                                   n_iter=10,
                                   cv=3,
                                   scoring='neg_mean_squared_error',
                                   verbose=2,
                                   random_state=42)

# Train the model with the best hyperparameters
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.0059 - mean_squared_error: 0.0059
Epoch 2/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0012 - mean_squared_error: 0.0012
Epoch 3/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 9.4775e-04 - mean_squared_error: 9.4775e-04
Epoch 4/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 7.1345e-04 - mean_squared_error: 7.1345e-04
Epoch 5/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 8.7064e-04 - mean_squared_error: 8.7064e-04
Epoch 6/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 7.7365e-04 - mean_squared_error: 7.7365e-04
Epoch 7/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 6.4392e-04 - mean_squared_error: 6.4392e-04
Epoch 8/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 6.5295e-04 - mean_squared_error: 6.5295e-04
Epoch 9/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 7.7156e-04 - mean_squared_error: 7.7156e-04
Epoch 10/80
33/33 ━━━━━━━━━━

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 7.1255e-04 - mean_squared_error: 7.1255e-04
Epoch 61/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 4.3289e-04 - mean_squared_error: 4.3289e-04
Epoch 62/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 6.3151e-04 - mean_squared_error: 6.3151e-04
Epoch 63/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 4.4158e-04 - mean_squared_error: 4.4158e-04
Epoch 64/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 4.3158e-04 - mean_squared_error: 4.3158e-04
Epoch 65/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 3.8534e-04 - mean_squared_error: 3.8534e-04
Epoch 66/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 4.6164e-04 - mean_squared_error: 4.6164e-04
Epoch 67/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 4.0047e-04 - mean_squared_error: 4.0047e-04
Epoch 68/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 3.6542e-04 - mean_squared_error: 3.6542e-04
Epoch 69/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 4.3126e-04 - mea

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 2.7585e-04 - mean_squared_error: 2.7585e-04
Epoch 39/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 2.9391e-04 - mean_squared_error: 2.9391e-04
Epoch 40/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 2.7769e-04 - mean_squared_error: 2.7769e-04
Epoch 41/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 2.8678e-04 - mean_squared_error: 2.8678e-04
Epoch 42/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 2.9332e-04 - mean_squared_error: 2.9332e-04
Epoch 43/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 2.7730e-04 - mean_squared_error: 2.7730e-04
Epoch 44/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 2.8703e-04 - mean_squared_error: 2.8703e-04
Epoch 45/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 2.6503e-04 - mean_squared_error: 2.6503e-04
Epoch 46/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 2.7531e-04 - mean_squared_error: 2.7531e-04
Epoch 47/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 2.7307e-04 - mea

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 3.9153e-04 - mean_squared_error: 3.9153e-04
Epoch 17/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 3.9111e-04 - mean_squared_error: 3.9111e-04
Epoch 18/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 4.2634e-04 - mean_squared_error: 4.2634e-04
Epoch 19/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 5.2428e-04 - mean_squared_error: 5.2428e-04
Epoch 20/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 4.8033e-04 - mean_squared_error: 4.8033e-04
Epoch 21/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 4.7787e-04 - mean_squared_error: 4.7787e-04
Epoch 22/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 4.8997e-04 - mean_squared_error: 4.8997e-04
Epoch 23/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 3.8663e-04 - mean_squared_error: 3.8663e-04
Epoch 24/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 3.9388e-04 - mean_squared_error: 3.9388e-04
Epoch 25/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 4.0215e-04 - mea

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 4.2229e-04 - mean_squared_error: 4.2229e-04
Epoch 76/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 3.7466e-04 - mean_squared_error: 3.7466e-04
Epoch 77/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 4.3264e-04 - mean_squared_error: 4.3264e-04
Epoch 78/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 3.9612e-04 - mean_squared_error: 3.9612e-04
Epoch 79/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 5.3012e-04 - mean_squared_error: 5.3012e-04
Epoch 80/80
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 4.2287e-04 - mean_squared_error: 4.2287e-04
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
[CV] END batch_size=64, dropout_rate=0.2, epochs=80, learning_rate=0.01, optimizer=adam, units=400; total time= 1.3min
Epoch 1/80
130/130 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.0029 - mean_squared_error: 0.0029
Epoch 2/80
130/130 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 7.9208e-04 - mean_squared_error: 7.9208e-04
Epoch 3/80
130/130 ━━━━━━━━

KeyboardInterrupt: 

In [ ]:
# Output the best hyperparameter
print("Beste Hyperparameter:", random_search.best_params_)